# Introduction
Hey, thanks for viewing my Kernel!

If you like my work, please, leave an upvote: it will be really appreciated and it will motivate me in offering more content to the Kaggle community ! 😊

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

# For exploring audio files
import librosa
import librosa.display
import IPython.display as ipd

sns.set()
BASE_DIR = '../input/birdclef-2022/'
train = pd.read_csv(f'{BASE_DIR}/train_metadata.csv')
test = pd.read_csv(f'{BASE_DIR}/test.csv')
ebird = pd.read_csv(f'{BASE_DIR}/eBird_Taxonomy_v2021.csv')
ss = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

train.head()

In [ ]:
train.isna().sum()

In [ ]:
train['time_dt'] = pd.to_datetime(train['time'], errors='coerce')
train['time_dt'] = train['time_dt'].dt.round('30min')
train['time_H_M'] = train['time_dt'].dt.strftime('%H:%M')
print('Error Times :', train['time_dt'].isna().sum())

In [ ]:
train.loc[train['time_dt'].isna(), ['time', 'time_H_M']].head()

In [ ]:
train.dtypes

In [ ]:
train.duplicated().sum()

In [ ]:
def seperate_list_col(str_list, max_len=6):
    if str_list != '[]':
        seperated_list = str_list.split(',')
        seperated_list[0] = seperated_list[0][1:]
        seperated_list[-1] = seperated_list[-1][:-1]
        seperated_list.extend([None] * (6 - len(seperated_list)))
        return seperated_list
    else:
        return [None] * max_len

In [ ]:
train['secondary_label_len'] = train['secondary_labels'].apply(lambda x: len(x.split(',')))
train['type_len'] = train['type'].apply(lambda x: len(x.split(',')))

In [ ]:
values_secondary_labels = train['secondary_labels'].apply(lambda x: seperate_list_col(x, max_len=6))
df_secondary_labels = pd.DataFrame(values_secondary_labels.to_list(), columns=['l2', 'l3', 'l4', 'l5', 'l6', 'l7'])
df_secondary_labels.head()

In [ ]:
train['secondary_label_len'] = np.where(df_secondary_labels['l2'].values != None, train['secondary_label_len'], 0)

In [ ]:
values_type = train['type'].apply(lambda x: seperate_list_col(x, max_len=9))
df_type = pd.DataFrame(values_type.to_list(), columns=['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9'])
df_type.head()

In [ ]:
train['type_len'] = np.where(df_type['t1'].values != None, train['type_len'], 0)

# Distributions

In [ ]:
fig, ax = plt.subplots(figsize=(24, 8))
sns.countplot(data=train, x='common_name', ax=ax, order=train['common_name'].value_counts().index)
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(data=train, x='rating', ax=ax);

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
x_labels = pd.date_range(start='00:00', periods=48, freq='30min')
x_labels = list(x_labels.strftime('%H:%M'))
sns.countplot(data=train, x='time_H_M', ax=ax)
ax.set_xticklabels(x_labels, rotation=90);

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(data=train, x='secondary_label_len', ax=ax);

In [ ]:
values = [df_secondary_labels[col].values.tolist() for col in df_secondary_labels.columns]
values = np.array(values)
df_values = pd.DataFrame(values.flatten(), columns=['secondary_labels'])

fig, ax = plt.subplots(figsize=(24, 8))
sns.countplot(data = df_values, x='secondary_labels', ax=ax, order=df_values['secondary_labels'].value_counts().index)
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(data=train, x='type_len', ax=ax);

In [ ]:
values = [df_type[col].values.tolist() for col in df_type.columns]
values = np.array(values)
df_values = pd.DataFrame(values.flatten(), columns=['df_type'])
th = 10
counts = df_values['df_type'].value_counts()
ignore_values = counts[counts < th].index
df_values.loc[df_values['df_type'].isin(ignore_values), 'df_type'] = 'OTHERS'

fig, ax = plt.subplots(figsize=(24, 8))
sns.countplot(data = df_values, x='df_type', ax=ax, order=df_values['df_type'].value_counts().index)
plt.xticks(rotation=90);

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(data=train, x='license', ax=ax)
plt.xticks(rotation=45);

In [ ]:
fig = px.scatter_geo(
    train,
    lat="latitude",
    lon="longitude",
    color="common_name",
    width=1_000,
    height=500,
    title="BirdCLEF 2022 Training Data",
)
fig.show()

# Noise Reduction

"Noise reduction is the process of removing noise from a signal. Noise reduction techniques exist for audio and images. Noise reduction algorithms may distort the signal to some degree." [Source](https://en.wikipedia.org/wiki/Noise_reduction)

In this work, we use a low pass filter for noise reduction. We can control noise reduction rate by changin **"th"** parameter.

In [ ]:
# Listen to the audio for the second training example
fn = train["filename"].values[1]
ipd.Audio(f"{BASE_DIR}train_audio/{fn}")

In [ ]:
y, sr = librosa.load(f"{BASE_DIR}train_audio/{fn}")
fig, ax = plt.subplots(figsize=(16, 8))
sns.lineplot(x=np.arange(len(y)), y=y, ax=ax);

In [ ]:
def noise_reduction(y, sr, plot=True, th=0.3):
    from scipy.fft import fft, fftfreq, ifft
    
    SAMPLE_RATE = 1
    DURATION = len(y) / SAMPLE_RATE
    N = int(SAMPLE_RATE * DURATION)

    yf = fft(y)
    xf = fftfreq(N, 1 / SAMPLE_RATE)
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(24, 8))
        axes[0].plot(np.arange(len(y)), y)
        axes[0].set_title('Before Time-Domain')
        axes[1].plot(xf, np.abs(yf))
        axes[1].set_title('Before Frequency-Domain')
        plt.show()
    
    # Filtering Low-Pass
    new_yf = yf.copy()
    middle = len(y) / 2
    new_yf[int(middle - len(y) * th):int(middle + len(y) * th)] = 0
    new_y = ifft(new_yf)
    new_y = new_y.real
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(24, 8))
        axes[0].plot(np.arange(len(y)), new_y)
        axes[0].set_title('After Time-Domain')
        axes[1].plot(xf, np.abs(new_yf))
        axes[1].set_title('After Frequency-Domain')
        plt.show()
    
    return new_y, sr

In [ ]:
new_y, sr = noise_reduction(y, sr, plot=True, th=0.3)

In [ ]:
display("Old Audio", ipd.Audio(data=y, rate=sr))
display("New Audio", ipd.Audio(data=new_y, rate=sr))